In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from scipy import signal
from matplotlib import pyplot as plt
from PIL import Image
#from joblib import Parallel, delayed  #multithreading 

In [2]:
# Definir una red neuronal simple
class Model_qm(nn.Module):
    def __init__(self):
        super(Model_qm, self).__init__()
        self.fc1 = nn.Linear(8, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 8)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [3]:
q_m_XOR = Model_qm()
state_dict_XOR = torch.load('q_m_XOR.pth')
q_m_XOR.load_state_dict(state_dict_XOR)

def q_m_XOR_diff(bits):
    return q_m_XOR(bits)

In [4]:
q_m_XNOR = Model_qm()
state_dict_XNOR = torch.load('q_m_XNOR.pth')
q_m_XNOR.load_state_dict(state_dict_XNOR)

def q_m_XNOR_diff(bits):
    return q_m_XNOR(bits)

In [41]:
def q_m_diff(input):
    output = torch.zeros((input.shape[0],9), dtype=torch.float32)
    all_ind_mask = torch.ones(input.shape[0], dtype=torch.bool)
    num_1 = (input > 0.5).sum(dim=1)
    #print(f"num_1:{num_1}")
    num_1_plus4_mask = (num_1 > 4) # mask donde la suma de 1 es mayor a 4
    #print(f"num_1_plus4_mask: {num_1_plus4_mask}")
    num_1_equal4_mask = (num_1 == 4) # mask donde la suma de 1 es 4
    #print(f"num_1_equal4_mask:{num_1_equal4_mask}")                      
    data0_equal0_mask = (input[:,0] == 0) # mask donde el primer valor de la entrada es 0
    #print(f"data0_equal0_mask:{data0_equal0_mask}")  
    intersect_ind_mask = (data0_equal0_mask & num_1_equal4_mask) # mask donde el primer valor de la entrada es 0 y la suma de 1 es 4
    #print(f"intersect_ind_mask 1:{intersect_ind_mask}")   
    intersect_ind_mask = (intersect_ind_mask | num_1_plus4_mask)  # mask donde: (el primer valor de la entrada es 0 y la suma de 1 es 4) OR (la suma es mayor a 4)
    #print(f"intersect_ind_mask 2:{intersect_ind_mask}")   
    intersect_ind = intersect_ind_mask.nonzero().squeeze()  # posiciones donde: (el primer valor de la entrada es 0 y la suma de 1 es 4) OR (la suma es mayor a 4)
    #print(f"intersect_ind:{intersect_ind}")   
    rest_ind = (intersect_ind_mask != all_ind_mask).nonzero().squeeze() #resto de indices que no cumplen condiciones anteriores
    #print(f"rest_ind:{rest_ind}")

    output[intersect_ind,:8] = q_m_XNOR_diff(input[intersect_ind,:])
    output[intersect_ind,8] = 0
    output[rest_ind,:8] = q_m_XOR_diff(input[rest_ind,:])
    output[rest_ind,8] = 1

    return output

In [42]:
def TMDS_diff(pixel_column_bits,cnt_column):
    bits_inversos = torch.flip(pixel_column_bits, dims = (1,))  #ahora es en la dim = 1 y no 0
    q_m = q_m_diff(bits_inversos)
    #print(f"qm_diff:{torch.round(q_m)}")
    output = torch.zeros((pixel_column_bits.shape[0],10),dtype=torch.float32)
    num_1 = (q_m[:,:8] > 0.5).sum(dim = 1)
    num_0 = (q_m[:,:8] < 0.5).sum(dim = 1)
    #print(f"num1:{num_1}")
    #print(f"num0:{num_0}")
    all_ind_mask = torch.ones(q_m.shape[0], dtype=torch.bool)
    IndE_mask = (cnt_column == 0) | (num_1 == num_0)
    #print(f"IndE_mask:{IndE_mask}")
    IndE_ind = ((cnt_column == 0) | (num_1 == num_0)).nonzero().squeeze()
    #print(f"IndE_ind:{IndE_ind}")
    IndC_mask = ((cnt_column > 0) & (num_1 > num_0)) | ((cnt_column < 0) & (num_0 > num_1))
    #print(f"IndC_mask:{IndC_mask}")
    Neg_q = 1 - q_m
    #print(f"Neg_qm_diff:{torch.round(Neg_q)}")
    IndE_and_q_m_0 = ((IndE_mask) & (q_m[:,8] < 0.5)).nonzero().squeeze()
    #print(f"IndE_and_q_m_0:{IndE_and_q_m_0}")
    IndE_and_q_m_1 = ((IndE_mask) & (q_m[:,8] > 0.5)).nonzero().squeeze()
    #print(f"IndE_and_q_m_1:{IndE_and_q_m_1}")
    NotIndE_and_IndC = (torch.logical_not(IndE_mask) & IndC_mask).nonzero().squeeze()
    #print(f"NotIndE_and_IndC:{NotIndE_and_IndC}")
    NotIndE_and_NotIndC = (torch.logical_not(IndE_mask) & torch.logical_not(IndC_mask)).nonzero().squeeze()
    #print(f"NotIndE_and_NotIndC:{NotIndE_and_NotIndC}")
    q_m_mask = (IndE_mask & (q_m[:,8] > 0.5)) | (torch.logical_not(IndC_mask) & torch.logical_not(IndE_mask))
    #print(f"q_m_mask:{q_m_mask}")
    q_m_ind = ((IndE_mask & (q_m[:,8] > 0.5)) | (torch.logical_not(IndC_mask) & torch.logical_not(IndE_mask))).nonzero().squeeze()
    #print(f"q_m_ind:{q_m_ind}")
    Neg_q_ind = (q_m_mask != all_ind_mask).nonzero().squeeze()
    #print(f"Neg_q_ind:{Neg_q_ind}")
    output[q_m_ind,:8] = q_m[q_m_ind,:8]
    output[Neg_q_ind,:8] = Neg_q[Neg_q_ind,:8]
    output[:,8] = q_m[:,8]
    output[IndE_ind,9] = Neg_q[IndE_ind,8]
    new_cnt = cnt_column.clone()
    new_cnt[IndE_and_q_m_0] = cnt_column[IndE_and_q_m_0] + num_0[IndE_and_q_m_0] - num_1[IndE_and_q_m_0]
    new_cnt[IndE_and_q_m_1] = cnt_column[IndE_and_q_m_1] + num_1[IndE_and_q_m_1] - num_0[IndE_and_q_m_1]
    #print(f"new_cnt:{new_cnt}")
    output[NotIndE_and_IndC,9] = 1
    new_cnt[NotIndE_and_IndC] = cnt_column[NotIndE_and_IndC] + 2 * q_m[NotIndE_and_IndC,8] + num_0[NotIndE_and_IndC] - num_1[NotIndE_and_IndC]
    #print(f"new_cnt:{new_cnt}")
    output[NotIndE_and_NotIndC,9] = 0
    new_cnt[NotIndE_and_NotIndC] = cnt_column[NotIndE_and_NotIndC] - 2 * Neg_q[NotIndE_and_NotIndC,8] + num_1[NotIndE_and_NotIndC] - num_0[NotIndE_and_NotIndC]
    #print(f"new_cnt:{new_cnt}")

    return output,new_cnt

In [32]:
batch = torch.zeros((2,8))
batch[0,:] = torch.tensor([0, 0, 0, 0, 0, 0, 0, 1])
batch[1,:] = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0])
cnt_column = torch.zeros(2)
cnt_column[0] = -9

In [39]:
out,cnt = TMDS_diff(batch,cnt_column)
print(torch.round(out))
print(torch.round(cnt))

num_1:tensor([1, 0])
num_1_plus4_mask: tensor([False, False])
num_1_equal4_mask:tensor([False, False])
data0_equal0_mask:tensor([False,  True])
intersect_ind_mask 1:tensor([False, False])
intersect_ind_mask 2:tensor([False, False])
intersect_ind:tensor([], dtype=torch.int64)
rest_ind:tensor([0, 1])
qm_diff:tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.]], grad_fn=<RoundBackward0>)
num1:tensor([8, 0])
num0:tensor([0, 8])
IndE_mask:tensor([False,  True])
IndE_ind:1
IndC_mask:tensor([False, False])
Neg_qm_diff:tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0.]], grad_fn=<RoundBackward0>)
IndE_and_q_m_0:tensor([], dtype=torch.int64)
IndE_and_q_m_1:1
NotIndE_and_IndC:tensor([], dtype=torch.int64)
NotIndE_and_NotIndC:0
q_m_mask:tensor([True, True])
q_m_ind:tensor([0, 1])
Neg_q_ind:tensor([], dtype=torch.int64)
new_cnt:tensor([-9., -8.])
new_cnt:tensor([-9., -8.], grad_fn=<IndexPutBackward0>)
new_cnt:tensor([-1

In [9]:
def sigmoid(x):
    x_in = x.clone()
    if x >= 0:
        return 1 / (1 + torch.exp(-x_in))
    else:
        return torch.exp(x_in) / (1 + torch.exp(x_in))

def Pixel2Bit_diff(pixel):
    output = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype= torch.float32)
    for i in range(1,9):
        output[i-1] = sigmoid(10*(pixel-2**(8-i)+0.5))  # 0.5 para ajustar la sigmoidal
        if pixel >= 2**(8-i):
            pixel = pixel - 2**(8-i)
    return output


In [10]:
#funcion que toma como entrada el armonico a sintonizar y las dimensiones de la imagen a espiar y devuelve un array con taps de g(t)
def g_taps(dim_vertical, dim_horizontal, armonico):

    #defino variables iniciales
    f_b = 10 * (dim_vertical * dim_horizontal * 60)
    f_sdr = 50e6
    harm = armonico * f_b
    
    #para el correcto funcionamiento: dependiendo del armonico, elijo cuantas muestras por pulso
    if (armonico < 5 ):
        muestras_por_pulso  = 10
    else:
        muestras_por_pulso  = 20

    samp_rate = muestras_por_pulso * f_b
    H_samples = dim_horizontal * muestras_por_pulso

    #creo el pulso
    t_continuous = np.linspace(start = 0, stop = H_samples/samp_rate, num = H_samples, endpoint= False)
    pulso = np.zeros(H_samples)
    pulso[:muestras_por_pulso] = 1

    #traslado el espectro del pulso el armonico correspondiente
    frec_armonico = np.exp(-2j*np.pi*harm*t_continuous)
    pulso_complejo = pulso*frec_armonico

    #creo el lpf del sdr
    b, a = signal.butter(6, f_sdr/2, fs=samp_rate, btype='lowpass', analog=False)

    #filtro con lpf el pulso multiplicado por armonico. El resultado es g
    g_t = signal.lfilter(b, a, pulso_complejo)
    g_t = signal.decimate(g_t,q = muestras_por_pulso)

    # si armonico crece, necesito mas taps
    if (armonico < 5):
        g_t = g_t[:30]
    else:
        g_t = g_t[:200]

    g_t_max = np.max(np.abs(g_t))
 
    g_t = g_t / g_t_max

    return torch.tensor(g_t,dtype = torch.complex64).reshape(1,1,len(g_t))

In [11]:
def forward(filas,columnas,g_t,padding,img_slice):
    cnt_column = torch.zeros(filas)
    pixels_column = torch.zeros(filas)
    pixel_column_bits = torch.zeros((img_slice[0],8),dtype = torch.float32)
    bits_cod_fila =  torch.zeros((img_slice.shape[0],1,10*columnas), dtype = torch.complex64)
    for j in range(columnas):
        pixels_column = img_slice[:,j]
        pixel_column_bits = Pixel2Bit_diff(pixels_column)
        bits_cod_fila[:,:,j*10:(j+1)*10],cnt_column[:] = TMDS_diff(pixel_column_bits, cnt_column)
    img_slice_out = nn.functional.conv1d(bits_cod_fila, g_t, stride = 10, padding=padding, bias = None)[:,:,:].reshape(filas,columnas)
    return img_slice_out

In [ ]:
image_path = '../../images/VAMO!!.png'
img = np.asarray(Image.open(image_path))[:,:,0] #solo canal rojo
img_slice = torch.tensor(img[:10,:], dtype=torch.float32, requires_grad=True)
armonico = 3
g_t = g_taps(img.shape, armonico)
size_g_t = g_t.numel()    
padding = (size_g_t - 10)//2
imagen_fila_salida = forward(250,g_t,padding,img_slice)

In [12]:
def uint8_to_binarray(integer):
  """Convert integer into fixed-length 8-bit binary array. LSB in [0].
  Extended and modified code from https://github.com/projf/display_controller/blob/master/model/tmds.py
  """

  b_array = [int(i) for i in reversed(bin(integer)[2:])]
  b_array += [0]*(8-len(b_array))
  return b_array


def binarray_to_uint(binarray):
	
  array = binarray[::-1]
  num = array[0]
  for n in range(1,len(binarray)):
    num = (num << 1) + array[n]

  return num

def TMDS_pixel (D,cnt=0):
  """8bit pixel TMDS coding

  Inputs: 
  - pix: 8-bit pixel
  - cnt: 0's and 1's balance. Default in 0 (balanced)

  Outputs:
  - pix_out: TDMS coded 16-bit pixel (only 10 useful)
  - cnt: 0's and 1's balance updated with new pixel coding

  """ 
  # Convert 8-bit pixel to binary list D
  #D = uint8_to_binarray(pix)

  # Initialize output q
  qm = [D[0]]

  # 1's unbalanced condition at current pixel
  N1_D = np.sum(D)

  if N1_D>4 or (N1_D==4 and not(D[0])):

    # XNOR of consecutive bits
    for k in range(1,8):
      qm.append( not(qm[k-1] ^ D[k]) )
    qm.append(0)

  else:
    # XOR of consecutive bits
    for k in range(1,8):
      qm.append( qm[k-1] ^ D[k] )
    qm.append(1)

  # Initialize output qout
  qout = qm.copy()

  # Unbalanced condition with previous and current pixels
  N1_qm = np.sum(qm[:8])
  N0_qm = 8 - N1_qm

  if cnt==0 or N1_qm==4:

    qout.append(not(qm[8]))
    qout[8] = qm[8]
    qout[:8]=qm[:8] if qm[8] else np.logical_not(qm[:8])

    if not(qm[8]):
      cnt += N0_qm - N1_qm 
    else:
      cnt += N1_qm - N0_qm 

  else:

    if (cnt>0 and N1_qm>4) or (cnt<0 and N1_qm<4):
      qout.append(1)
      qout[8] = qm[8]
      qout[:8] = np.logical_not(qm[:8])
      cnt += 2*qm[8] + N0_qm - N1_qm
    else:
      qout.append(0)
      qout[8] = qm[8]
      qout[:8] = qm[:8]
      cnt += -2*(not(qm[8])) + N1_qm - N0_qm

  return qout, cnt

In [43]:
import itertools
enteros = range(-9, 10)
binarios = [list(map(int, list(bin(i)[2:].zfill(8)))) for i in range(256)]

combinaciones = list(itertools.product(binarios, enteros))

X = np.array([list(comb[0]) + [comb[1]] for comb in combinaciones])
X_t = torch.tensor(X,dtype=torch.float32)

print(X)
print(X_t)
print(X.shape)
print(X_t.shape)

[[ 0  0  0 ...  0  0 -9]
 [ 0  0  0 ...  0  0 -8]
 [ 0  0  0 ...  0  0 -7]
 ...
 [ 1  1  1 ...  1  1  7]
 [ 1  1  1 ...  1  1  8]
 [ 1  1  1 ...  1  1  9]]
tensor([[ 0.,  0.,  0.,  ...,  0.,  0., -9.],
        [ 0.,  0.,  0.,  ...,  0.,  0., -8.],
        [ 0.,  0.,  0.,  ...,  0.,  0., -7.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  7.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  8.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  9.]])
(4864, 9)
torch.Size([4864, 9])


In [44]:
errores = 0
bits_diff = torch.zeros((X.shape[0],10))
cnt_diff = torch.zeros((X.shape[0],1))
bits_diff,cnt_diff = TMDS_diff(X_t[:,:8],X_t[:,8])
for i in range(X.shape[0]):
    bits_posta,cnt_posta = TMDS_pixel(X[i,:8][::-1],X[i,8])
    bits_posta = np.array(bits_posta)
    if not(np.array_equal(np.round(bits_diff[i,:].detach().numpy()), bits_posta[:10])) and cnt_posta != np.round(cnt_diff[i].detach().numpy()):
        errores = errores + 1
        print(f"input: {X[i,:8]} cnt:{X[i,8]}")
        print(f"diff:{np.round(bits_diff[i,:].detach().numpy())} cnt diff:{cnt_diff[i].detach().numpy()}")
        print(f"posta:{bits_posta},cnt:{cnt_posta}")
        print(f"\n\n")
print(errores)

0
